In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pandas_ta as ta
import datetime as dt
import requests
import time
import schedule
from Paper_Portfolio import Portfolio 
import websockets
import json
import asyncio
import nest_asyncio
nest_asyncio.apply()

In [2]:
# Get Historical Candles: Excluding Real-Time Bar
def get_ohlc_data():
    # Define the endpoint URL
    url = "https://api.cryptowat.ch/markets/kraken/btcusd/ohlc"
    
    # Define the parameters
    params = {
        "periods": 900,  # 15 minutes in seconds
        "after": int(time.time()) - 900 * 1600,  # Get the last 3000 15-minute periods
    }
    
    # Send the GET request
    response = requests.get(url, params=params)
    
    # Convert the response to JSON
    data = response.json()
    
    # Get the OHLC data
    ohlc_data = data["result"]["900"]
    
    # Convert the OHLC data to a pandas DataFrame
    df = pd.DataFrame(ohlc_data, columns=["Time", "Open", "High", "Low", "Close", "Volume", "QuoteVolume"])
    df = df.drop(columns=['Volume', 'QuoteVolume'])
    # Convert the CloseTime to a datetime
    df["Time"] = pd.to_datetime(df["Time"], unit="s")
    
    return df

# Use the function
data = get_ohlc_data()
data['hlc3'] = (data['High'] + data['Low'] + data['Close']) / 3
data.set_index('Time', inplace=True)
data = data.shift(-1)
data = data.drop(data.tail(1).index)
data = data.drop(data.tail(1).index)
print(data)

                        Open     High      Low    Close          hlc3
Time                                                                 
2023-08-03 01:15:00  29168.8  29168.8  29153.2  29163.6  29161.866667
2023-08-03 01:30:00  29163.6  29175.1  29163.6  29175.1  29171.266667
2023-08-03 01:45:00  29175.1  29249.8  29175.1  29237.3  29220.733333
2023-08-03 02:00:00  29237.3  29237.5  29216.1  29216.1  29223.233333
2023-08-03 02:15:00  29216.1  29216.2  29154.6  29154.6  29175.133333
...                      ...      ...      ...      ...           ...
2023-08-19 15:45:00  26058.7  26210.0  26058.7  26210.0  26159.566667
2023-08-19 16:00:00  26210.0  26210.0  26113.1  26126.7  26149.933333
2023-08-19 16:15:00  26126.7  26177.3  26125.1  26139.0  26147.133333
2023-08-19 16:30:00  26139.0  26139.0  26119.0  26139.0  26132.333333
2023-08-19 16:45:00  26139.0  26250.0  26139.0  26246.2  26211.733333

[1597 rows x 5 columns]


In [3]:
# Helper Functions

def normalize(series):
    # Reshape the data
    series = pd.Series()
    series = series.values.reshape(-1, 1)
    # Fit and transform the data
    series_scaled = scaler.fit_transform(series)
    return pd.Series(series_scaled.flatten())

def rescale(series, min_val, max_val, scale_min, scale_max):
    return [(value - min_val) / (max_val - min_val) * (scale_max - scale_min) + scale_min if not np.isnan(value) else np.nan for value in series]

def get_lorentzian_distance(i, j, featureDataFrame):
    distance = 0
    for feature in featureDataFrame.columns:
        distance += np.log(1 + np.abs(featureDataFrame[feature].iloc[j] - featureDataFrame[feature].iloc[i]))
    return distance


def series_from(feature_string, f_paramA, f_paramB, data):
    switcher = {
        "RSI": n_rsi(f_paramA, f_paramB, data),
        "WT": n_wt(f_paramA, f_paramB, data),
        "CCI": n_cci(f_paramA, f_paramB, data),
    }
    return switcher.get(feature_string)

def get_feature_df(feature_params, data):
    feature_df = pd.DataFrame()
    for feature in feature_params.keys():
        feature_type = feature_params[feature][0]
        param_a = feature_params[feature][1]
        param_b = feature_params[feature][2]
        feature_df[feature] = series_from(feature_type, param_a, param_b, data)
    feature_df.index = data.index
    return feature_df

def get_prediction(data, feature_df, y_train, neighors, max_bars_back):
    # Initialize arrays
    global predictions_all
    global distances_all
    
    # Initialize lastDistance
    lastDistance = -1

    count = -1
    # Begin from the max_bars_back index
    for j in range(len(data)-max_bars_back, len(data)-1): #len(data)-1
        d = get_lorentzian_distance(j, len(data)-1, feature_df)
        count +=1
        if d >= lastDistance and count%4 == 0:
            lastDistance = d
            distances_all.append(d)
            predictions_all.append(round(y_train[j]))
            if len(predictions_all) > neighbors:
                lastDistance = np.percentile(distances_all, 25)
                distances_all = distances_all[1:]
                predictions_all = predictions_all[1:]
        prediction = sum(predictions_all)
    return prediction

def get_decision(signal, data):
    
    # Create a series that is 1 when the signal is positive and 0 when it's negative
    signs = np.sign(signal)
    signs = (signs > 0).astype(int)

    # Create a series that increments when the sign changes
    groups = signs.diff().ne(0).cumsum()

    # Count consecutive positive or negative values
    barsHeld = groups.groupby(groups).cumcount() + 1

    # Create isHeldFourBars
    isHeldFourBars = barsHeld >= 4

    # Create isHeldLessThanFourBars
    isHeldLessThanFourBars = (0 < barsHeld) & (barsHeld < 4)
    
    # Create isDifferentSignalType
    isDifferentSignalType = signal.diff() != 0

    # Create isEarlySignalFlip
    isEarlySignalFlip = isDifferentSignalType & ((signal.shift(1).diff() != 0) | (signal.shift(2).diff() != 0) | (signal.shift(3).diff() != 0))

    # Create isBuySignal
    isBuySignal = signal == 1

    # Create isSellSignal
    isSellSignal = signal == -1

    # Create isLastSignalBuy
    isLastSignalBuy = signal.shift(4) == 1

    # Create isLastSignalSell
    isLastSignalSell = signal.shift(4) == -1

    # Create isLastSignalBuy
    isLastSignalBuy = ((signal.shift(4) == 1) & (signal.shift(3) == 1) & (signal.shift(3) == 1) & (signal.shift(1) == 1))

    # Create isLastSignalSell
    isLastSignalSell = ((signal.shift(4) == -1) & (signal.shift(3) == -1) & (signal.shift(3) == -1) & (signal.shift(1) == -1))
    
    # Create isNewBuySignal
    isNewBuySignal = isBuySignal & isDifferentSignalType

    # Create isNewSellSignal
    isNewSellSignal = isSellSignal & isDifferentSignalType
    
    # Create endLongTradeStrict
    endLongTradeStrict = (((isHeldFourBars & isLastSignalBuy) | (isNewSellSignal & isLastSignalBuy)) & isNewBuySignal.shift(4)) | (isEarlySignalFlip & ~isNewBuySignal)

    # Create endShortTradeStrict
    endShortTradeStrict = (((isHeldFourBars & isLastSignalSell) | (isNewBuySignal & isLastSignalSell)) & isNewSellSignal.shift(4)) | (isEarlySignalFlip & ~isNewSellSignal)
    
    # Filter the series
    buy_filtered = isNewBuySignal[isNewBuySignal]
    sell_filtered = isNewSellSignal[isNewSellSignal]
    buy_exit_filtered = endLongTradeStrict[endLongTradeStrict]
    sell_exit_filtered = endShortTradeStrict[endShortTradeStrict]
    
    buy_filtered = pd.Series(buy_filtered)
    sell_filtered = pd.Series(sell_filtered)
    buy_exit_filtered = pd.Series(buy_exit_filtered)
    sell_exit_filtered = pd.Series(sell_exit_filtered)
    buy_df = pd.concat([buy_filtered, buy_exit_filtered], axis=1)
    mask = buy_df[1] != buy_df[1].shift(1)

    # Apply the mask to the dataframe
    buy_df = buy_df[mask]
    sell_df = pd.concat([sell_filtered, sell_exit_filtered], axis=1)
    mask = sell_df[1] != sell_df[1].shift(1)
    sell_df = sell_df[mask]
    #print(buy_df.tail(5))
    #print(sell_df.tail(5))
    
    last_index = data.index[-1]
    
    if last_index in buy_df.index and last_index in sell_df.index:
        # Early Signal Flip
        if buy_df.loc[last_index, 0] == True:
            return 3
        elif sell_df.loc[last_index, 0] == True:
            return -3
    
    elif last_index in buy_df.index:
        # If the index is in buy_df, check if it's an entry or exit signal
        if buy_df.loc[last_index, 0] == True:  # Check if the value is True
            return 1  # Enter long
        else:
            return 2  # Exit long
    elif last_index in sell_df.index:
        # If the index is in sell_df, check if it's an entry or exit signal
        if sell_df.loc[last_index, 0] == True:  # Check if the value is True
            return -1  # Enter short
        else:
            return -2  # Exit short
    else:
        # If the index is not in either DataFrame, return 0
        return 0
    

In [4]:
# TA indicators

def n_wt(n1, n2, data):
    ema1 = ta.ema(data.hlc3, n1)
    ema2 = ta.ema(np.abs(data.hlc3 - ema1), n1)
    ci = (data.hlc3 - ema1) / (0.015 * ema2)
    wt1 = ta.ema(ci, n2)
    wt2 = ta.sma(wt1, 4)
    wt3 = wt1 - wt2
    max_val = np.nanmax(wt3)
    min_val = np.nanmin(wt3)
    normalized_series = rescale(wt3, min_val, max_val, 0, 1)
    normalized_series = pd.Series(normalized_series)
    normalized_expanding = (normalized_series - normalized_series.expanding().min()) / (normalized_series.expanding().max() - normalized_series.expanding().min())
    return normalized_expanding

def n_rsi(n1, n2, data):
    scaled = rescale(ta.ema(ta.rsi(data.Close, length=n1), n2), 0, 100, 0, 1)
    return scaled

def n_cci(n1, n2, data):
    ma = data.Close.rolling(window=n1).mean()
    cci = (data.Close - ma) / (0.015 * data.Close.rolling(window=n1).std(ddof=0))
    cci_ema = cci.ewm(span=n2, min_periods=n2, adjust=False).mean()
    normalized_series = (cci_ema - cci_ema.min()) / (cci_ema.max() - cci_ema.min())
    normalized_series = pd.Series(normalized_series)
    normalized_expanding = (normalized_series - normalized_series.expanding().min()) / (normalized_series.expanding().max() - normalized_series.expanding().min())
    return normalized_expanding


In [5]:
feature_params = {
    'f1': ['RSI', 14, 1],
    'f2': ['WT', 10, 11],
    'f3': ['RSI', 9, 1],
}

feature_df = get_feature_df(feature_params, data)
feature_df

,f1,f2,f3
Time,,,
2023-08-03 01:15:00,NaN,NaN,NaN
2023-08-03 01:30:00,NaN,NaN,NaN
2023-08-03 01:45:00,NaN,NaN,NaN
2023-08-03 02:00:00,NaN,NaN,NaN
2023-08-03 02:15:00,NaN,NaN,NaN
...,...,...,...
2023-08-19 15:45:00,0.781120,0.582190,0.840320
2023-08-19 16:00:00,0.650642,0.540915,0.661216
2023-08-19 16:15:00,0.659682,0.482612,0.672801


In [6]:
data_shifted = data.Close.shift(4)
y_train = pd.Series(np.zeros_like(data.Close), index = data.Close.index)
y_train[data.Close > data_shifted] = -1
y_train[data.Close < data_shifted] = 1
y_train

Time
2023-08-03 01:15:00    0.0
2023-08-03 01:30:00    0.0
2023-08-03 01:45:00    0.0
2023-08-03 02:00:00    0.0
2023-08-03 02:15:00    1.0
                      ... 
2023-08-19 15:45:00   -1.0
2023-08-19 16:00:00   -1.0
2023-08-19 16:15:00   -1.0
2023-08-19 16:30:00   -1.0
2023-08-19 16:45:00   -1.0
Length: 1597, dtype: float64

In [7]:
# Calculate Historical Data for prediction Series
max_bars_back = 1500
neighbors = 8
p_vals = []
prediction = 8
predictions_all = []
distances_all = []
for k in range(0, len(data)):
    if k >= max_bars_back-1:
        lastDistance = -1
        count = -1
        
        for j in range(k-max_bars_back+1, k):
            d = get_lorentzian_distance(j, k, feature_df)
            count += 1
            if d >= lastDistance and count%4 == 0:
                lastDistance = d
                distances_all.append(d)
                predictions_all.append(round(y_train[j]))
                if len(predictions_all) > neighbors:
                    lastDistance = np.percentile(distances_all, 25)
                    distances_all = distances_all[1:]
                    predictions_all = predictions_all[1:]
            prediction = sum(predictions_all)
    p_vals.append(prediction)

In [8]:
# Create a signal series based on prediction values
signal = []
for i in range(len(p_vals)):
    if p_vals[i] > 0:
        signal.append(1)
    elif p_vals[i] < 0:
        signal.append(-1)
    else:
        signal.append(signal[i-1])
signal = pd.Series(signal, index = data.Close.index)

In [9]:
# Create a series that is 1 when the signal is positive and 0 when it's negative
signs = np.sign(signal)
signs = (signs > 0).astype(int)

# Create a series that increments when the sign changes
groups = signs.diff().ne(0).cumsum()

# Count consecutive positive or negative values
barsHeld = groups.groupby(groups).cumcount() + 1

# Create isHeldFourBars
isHeldFourBars = barsHeld >= 4

# Create isHeldLessThanFourBars
isHeldLessThanFourBars = (0 < barsHeld) & (barsHeld < 4)

In [10]:
# Create isDifferentSignalType
isDifferentSignalType = signal.diff() != 0

# Create isEarlySignalFlip
isEarlySignalFlip = isDifferentSignalType & ((signal.shift(1).diff() != 0) | (signal.shift(2).diff() != 0) | (signal.shift(3).diff() != 0))

# Create isBuySignal
isBuySignal = signal == 1

# Create isSellSignal
isSellSignal = signal == -1

# Create isLastSignalBuy
isLastSignalBuy = ((signal.shift(4) == 1) & (signal.shift(3) == 1) & (signal.shift(3) == 1) & (signal.shift(1) == 1))

# Create isLastSignalSell
isLastSignalSell = ((signal.shift(4) == -1) & (signal.shift(3) == -1) & (signal.shift(3) == -1) & (signal.shift(1) == -1))

# Create isNewBuySignal
isNewBuySignal = isBuySignal & isDifferentSignalType

# Create isNewSellSignal
isNewSellSignal = isSellSignal & isDifferentSignalType

In [11]:
# Create endLongTradeStrict
endLongTradeStrict = (((isHeldFourBars & isLastSignalBuy) | (isNewSellSignal & isLastSignalBuy)) & isNewBuySignal.shift(4)) | (isEarlySignalFlip & ~isNewBuySignal)

# Create endShortTradeStrict
endShortTradeStrict = (((isHeldFourBars & isLastSignalSell) | (isNewBuySignal & isLastSignalSell)) & isNewSellSignal.shift(4)) | (isEarlySignalFlip & ~isNewSellSignal)

In [12]:
buy_filtered = isNewBuySignal[isNewBuySignal]
sell_filtered = isNewSellSignal[isNewSellSignal]
buy_exit_filtered = endLongTradeStrict[endLongTradeStrict]
sell_exit_filtered = endShortTradeStrict[endShortTradeStrict]

In [13]:
buy_filtered = pd.Series(buy_filtered)
sell_filtered = pd.Series(sell_filtered)
buy_exit_filtered = pd.Series(buy_exit_filtered)
sell_exit_filtered = pd.Series(sell_exit_filtered)
buy_df = pd.concat([buy_filtered, buy_exit_filtered], axis=1)
mask = buy_df[1] != buy_df[1].shift(1)

# Apply the mask to the dataframe
buy_df = buy_df[mask]
sell_df = pd.concat([sell_filtered, sell_exit_filtered], axis=1)
mask = sell_df[1] != sell_df[1].shift(1)
sell_df = sell_df[mask]

In [14]:
experiment_df = data.loc[buy_df.index, 'Close']
open_df = data.loc[buy_df.index, 'Open']
buy_df = pd.concat([buy_df, experiment_df], axis=1)
buy_df = pd.concat([buy_df, open_df], axis=1)
buy_df = buy_df.drop(buy_df.index[0])
buy_df = buy_df.drop(buy_df.index[0])
#buy_df.tail(50)

In [15]:
experiment_df2 = data.loc[sell_df.index, 'Close']
open_df2 = data.loc[sell_df.index, 'Open']
sell_df = pd.concat([sell_df, experiment_df2], axis=1)
sell_df = pd.concat([sell_df, open_df2], axis=1)
sell_df = sell_df.drop(sell_df.index[0])
#sell_df.tail(50)

In [16]:
# Calculate rough estimate of past Performance
df_odd_buy = buy_df['Close'].iloc[::2].reset_index(drop=True)
df_even_buy = buy_df['Open'].iloc[1::2].reset_index(drop=True)

df_odd_sell = sell_df['Close'].iloc[::2].reset_index(drop=True)
df_even_sell = sell_df['Open'].iloc[1::2].reset_index(drop=True)
# Calculate the percent change between the odd and even rows
pct_change_buy = (df_even_buy - df_odd_buy) / df_odd_buy
pct_change_sell = (df_even_sell - df_odd_sell) / df_odd_sell
pct_change_buy.sum()

-0.005230276343202423

In [17]:
# Main Trading Algorithm
portfolio = Portfolio(initial_balance=10000, leverage=1)
async def main():
    global portfolio
    message_count = -2
    in_long_trade = False
    in_short_trade = False
    prev_msg = None
    prev_signal = None
    prev_desc = None
    global data
    global feature_params
    global signal
    async with websockets.connect('wss://ws.kraken.com') as ws:
        # Send the subscription message
        await ws.send(json.dumps({
            "event": "subscribe",
            "pair": [
                "BTC/USD"
            ],
            "subscription": {
                "name": "ohlc",
                "interval": 15
            }
        }))
            
            
        async for msg in ws:
            message_count +=1
            info = json.loads(msg)

            # Ignore event messages
            if isinstance(info, dict):
                continue

            # Ignore the initial status message
            if len(info) == 4 and isinstance(info[1], str):
                continue

            ohlc_data = info[1]
            df = pd.DataFrame([ohlc_data])
            
            # Convert the OHLC data to a pandas DataFrame
            df = pd.DataFrame([ohlc_data], columns=['Extra', 'Time', 'Open', 'High', 'Low', 'Close', 'vwap', 'Volume', 'Count'])

            # Convert the Time to a datetime
            df["Time"] = pd.to_datetime(df["Time"], unit="s")

            # Set Time as the index
            df.set_index('Time', inplace=True)
            df = df.drop(columns = ['Extra','vwap','Volume','Count'])
            df = df.reindex(columns=['Open', 'High', 'Low', 'Close'])
            df = df.apply(pd.to_numeric, errors='coerce')
            df['hlc3'] = (df['High'] + df['Low'] + df['Close']) / 3

            if message_count == 1:
                prev_msg = df
                prev_desc = 0

            elif ((df.index[-1] != prev_msg.index[-1]) and (df['Open'].iloc[-1] != prev_msg['Open'].iloc[-1])):
                print('Is New Bar')
                print(df)
                data = pd.concat([data, prev_msg])
                data = data.drop(data.index[0])
                signal = pd.Series(signal, index = data.index)
                signal.iloc[-1] = prev_signal
                signal = signal.drop(signal.index[0])
                
                if (prev_desc == 1):
                    if (in_long_trade == False):
                        print("Enter Long")
                        print(df)
                        in_long_trade = True
                        portfolio.enter_long(price=df['Close'].iloc[-1], volume=0.1)

                elif (prev_desc == 2):
                    if (in_long_trade == True):
                        print("Exit Long")
                        print(df)
                        in_long_trade = False
                        portfolio.exit_long(price=df['Close'].iloc[-1], volume=0.1)

                elif (prev_desc == -1):
                    if (in_short_trade == False):
                        print("Enter Short")
                        print(df)
                        in_short_trade = True
                        portfolio.enter_short(price=df['Close'].iloc[-1], volume=0.1)

                elif (prev_desc == -2):
                    if (in_short_trade == True):
                        print("Exit Short")
                        print(df)
                        in_short_trade = False
                        portfolio.exit_short(price=df['Close'].iloc[-1], volume=0.1)

                elif (prev_desc == 3):
                    if (in_short_trade == True):
                        print("Early Long")
                        print(df)
                        in_short_trade = False
                        in_long_trade = True
                        portfolio.exit_short(price=df['Close'].iloc[-1], volume=0.1)
                        portfolio.enter_long(price=df['Close'].iloc[-1], volume=0.1)

                elif (prev_desc == -3):
                    if (in_long_trade == True):
                        print("Early Short")
                        print(df)
                        in_long_trade = False
                        in_short_trade = True
                        portfolio.exit_long(price=df['Close'].iloc[-1], volume=0.1)
                        portfolio.enter_short(price=df['Close'].iloc[-1], volume=0.1)

                
            data_real_time = pd.concat([data, df])
            data_real_time = data_real_time.drop(data_real_time.index[0])

            feature_df = get_feature_df(feature_params, data_real_time)

            data_real_time_shifted = data_real_time.Close.shift(4)
            y_train = pd.Series(np.zeros_like(data_real_time.Close), index = data_real_time.Close.index)
            y_train[data_real_time.Close > data_real_time_shifted] = -1
            y_train[data_real_time.Close < data_real_time_shifted] = 1
            prediction = get_prediction(data_real_time, feature_df, y_train, 8, 1500)

            signal_real_time = signal
            signal_real_time = signal_real_time.drop(signal_real_time.index[0])
            signal_real_time = pd.Series(signal_real_time, index = data_real_time.index)
            if prediction > 0:
                signal_real_time.iloc[-1] = 1
            elif prediction < 0:
                signal_real_time.iloc[-1] = -1
            else:
                signal_real_time.iloc[-1] = signal_real_time.iloc[-2]

            decision = get_decision(signal_real_time, data_real_time)
            #print(df)
            #print(decision)
            
            
            prev_desc = decision
            prev_signal = signal_real_time.iloc[-1]
            prev_msg = df

if __name__ == "__main__":
    loop = asyncio.get_event_loop()
    loop.run_until_complete(main())

Is New Bar
                        Open     High      Low    Close     hlc3
Time                                                            
2023-08-19 17:30:00  26133.1  26133.1  26133.1  26133.1  26133.1
Is New Bar
                        Open     High      Low    Close     hlc3
Time                                                            
2023-08-19 17:45:00  26081.3  26081.3  26081.3  26081.3  26081.3
Enter Short
                        Open     High      Low    Close     hlc3
Time                                                            
2023-08-19 17:45:00  26081.3  26081.3  26081.3  26081.3  26081.3
Is New Bar
                        Open     High      Low    Close     hlc3
Time                                                            
2023-08-19 18:00:00  26090.1  26090.1  26090.1  26090.1  26090.1
Is New Bar
                        Open     High      Low    Close     hlc3
Time                                                            
2023-08-19 18:15:00  26096.4  2609

ConnectionClosedError: no close frame received or sent

In [18]:
portfolio.calculate_profit()

5.379999999999928